In [3]:
import os
import pandas as pd
import numpy as np
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from wordcloud import WordCloud, STOPWORDS

# import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.naive_bayes import BernoulliNB


import numpy as np
import pandas as pd
# BeautifulSoup is used to remove html tags from the text
from bs4 import BeautifulSoup 
import re # For regular expressions

# Packages for data 
import pandas as pd
import numpy as np
import pickle
from collections import Counter

# Packages for machine learning modelling
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay, accuracy_score
# precision_score, recall_score, f1_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

# Packages for sentiment analysis
from textblob import TextBlob

# Packages for visualisation 
import matplotlib.pyplot as plt

# Packages for MLP
import nltk

import gensim
from gensim.models import word2vec
import gensim.downloader as api

In [5]:
# Load data

train_data = pd.read_csv("../Data/train_data.csv")
val_data = pd.read_csv("../Data/validation_data.csv")
test_data = pd.read_csv("../Data/test_data.csv")

X_train_text = train_data["text_preprocessed"].values
y_train = train_data["class_label"].values

X_val_text = val_data["text_preprocessed"].values
y_val = val_data["class_label"].values

X_test_text = test_data["text_preprocessed"].values
y_test = test_data["class_label"].values

In [6]:
# Create list of lists of Unigrams based on training corpus
lst_corpus = []
for string in X_train_text:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) 
                for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)

In [8]:
# Find mean length
num_articles = len(lst_corpus)
total_len = 0
for article in lst_corpus:
    total_len += len(article)

print('mean length =', total_len/num_articles)

mean length = 311.68333333333334


In [21]:
# Creating the model and setting values for the various parameters
corpus = lst_corpus # Must be a list of lists of tokens
num_features = 300  # Word vector dimensionality, i.e. target size of word vectors
min_word_count = 10 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 311        # Context window size, possible to use mean length of text in corpus
downsampling = 1e-3 # (0.001) Downsample setting for frequent words
skipgram = 1 # Use skipgram = 1

# Initializing the train model
model = word2vec.Word2Vec(
    sentences = corpus,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context,
    sample=downsampling,
    sg=skipgram
)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_10minwords_311context_skip"
model.save(model_name)
print("model saved")


model saved


# Load Model

In [39]:
# Load pre-trained Word2Vec model.
# model = gensim.models.Word2Vec.load('300features_40minwords_10context')
# model = gensim.models.Word2Vec.load('300features_10minwords_311context')
model = gensim.models.Word2Vec.load('Pretrained_Word2Vec/300features_10minwords_311context_skip')

In [23]:
words = list(model.wv.vocab)
print('length of vocab:', len(words))

length of vocab: 29926


In [24]:
# Example of similar words
w1="trump"
model.wv.most_similar(positive=w1)

[('donald', 0.9013704061508179),
 ('republican', 0.7219638824462891),
 ('presid', 0.7009391784667969),
 ('presidenti', 0.6926494836807251),
 ('campaign', 0.6864993572235107),
 ('support', 0.6612122058868408),
 ('candid', 0.6407409310340881),
 ('nomine', 0.601711630821228),
 ('democrat', 0.5973963141441345),
 ('obama', 0.5808975696563721)]

In [25]:
# Create word_vec dictionary
word_vec = dict({})
for idx, key, in enumerate(model.wv.vocab):
    word_vec[key] = model.wv[key]
# print("input word vectors:")
# print(word_vec)

### Generate document vectors

In [32]:
# Train
X_train = []
text = X_train_text
for i in range(len(text)):
    doc_vec_for_each_sent = np.array(300)
    for j in range(len(text[i])):
        word = text[i][j]
        if word in word_vec.keys():
            doc_vec_for_each_sent = np.add(doc_vec_for_each_sent,
                                            word_vec[word])
    doc_vec_for_each_sent = np.divide(doc_vec_for_each_sent, len(text[i]))
    X_train.append(doc_vec_for_each_sent)
# print('Generated Document Vectors:')
# print(X_train)
print(len(X_train))
print(len(text))

39060
39060


In [33]:
# Val
X_val = []
text = X_val_text
for i in range(len(text)):
    doc_vec_for_each_sent = np.array(300)
    for j in range(len(text[i])):
        word = text[i][j]
        if word in word_vec.keys():
            doc_vec_for_each_sent = np.add(doc_vec_for_each_sent,
                                            word_vec[word])
    doc_vec_for_each_sent = np.divide(doc_vec_for_each_sent, len(text[i]))
    X_val.append(doc_vec_for_each_sent)
# print('Generated Document Vectors:')
# print(X_val)
print(len(X_val))
print(len(text))

13020
13020


In [34]:
# Test
X_test = []
text = X_test_text
for i in range(len(text)):
    doc_vec_for_each_sent = np.array(300)
    for j in range(len(text[i])):
        word = text[i][j]
        if word in word_vec.keys():
            doc_vec_for_each_sent = np.add(doc_vec_for_each_sent,
                                            word_vec[word])
    doc_vec_for_each_sent = np.divide(doc_vec_for_each_sent, len(text[i]))
    X_test.append(doc_vec_for_each_sent)
# print('Generated Document Vectors:')
# print(X_test)
print(len(X_test))
print(len(text))

13021
13021


### Modelling

In [35]:
# Logreg
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_train, y_train)

y_val_pred = log_reg_clf.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.66      0.67      0.66      6361
           1       0.68      0.68      0.68      6659

    accuracy                           0.67     13020
   macro avg       0.67      0.67      0.67     13020
weighted avg       0.67      0.67      0.67     13020



In [38]:
# NB
naive_bayes_clf = BernoulliNB()
naive_bayes_clf.fit(X_train, y_train)

y_val_pred = naive_bayes_clf.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.49      0.97      0.65      6361
           1       0.60      0.04      0.08      6659

    accuracy                           0.50     13020
   macro avg       0.54      0.51      0.37     13020
weighted avg       0.55      0.50      0.36     13020



### Using pretrained Word2Vec